In [1]:

%pip install -q "openvino>=2023.1.0"
%pip install requests tqdm ipywidgets


import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import download_file

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openvino as ov

core = ov.Core()


In [3]:
devices = core.available_devices

for device in devices:
    device_name = core.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU: Intel(R) Xeon(R) Gold 6128 CPU @ 3.40GHz


In [4]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import os


base_url = 'https://storage.openvinotoolkit.org/repositories/open_model_zoo/2023.0/models_bin/1/'


models = {
   
    'face_detection': {
        'xml_file': 'face-detection-0200/FP32/face-detection-0200.xml',
        'bin_file': 'face-detection-0200/FP32/face-detection-0200.bin',
    }
}


save_dir = './model'


if not os.path.exists(save_dir):
    os.makedirs(save_dir)

def download_file(url, save_path):
    
    response = requests.get(url)
    response.raise_for_status() 

    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {save_path}")


for model in models.values():
   
    download_file(base_url + model['xml_file'], os.path.join(save_dir, os.path.basename(model['xml_file'])))
   
    download_file(base_url + model['bin_file'], os.path.join(save_dir, os.path.basename(model['bin_file'])))


Downloaded ./model/face-detection-0200.xml
Downloaded ./model/face-detection-0200.bin


In [9]:
import cv2
import numpy as np
from openvino.runtime import Core

def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

core = Core()
model_xml = "model/face-detection-0200.xml"
model = core.read_model(model=model_xml)
compiled_model = core.compile_model(model=model, device_name="CPU")

image_path = "testimages/7.jpg"
original_image = cv2.imread(image_path)
if original_image is None:
    raise FileNotFoundError(f"Could not load image at {image_path}")

input_shape = compiled_model.input().shape
image_resized = cv2.resize(original_image, (input_shape[3], input_shape[2]))
image_transposed = np.transpose(image_resized, (2, 0, 1))
image_input = np.expand_dims(image_transposed, 0)

# Calculate the average confidence threshold
confidence_thresholds = np.arange(0.3, 0.75, 0.05)
average_confidence_threshold = np.mean(confidence_thresholds)

result = compiled_model.infer_new_request({0: image_input})
output = result[compiled_model.outputs[0]]

detected_faces = []
for detection in output.reshape(-1, 7):
    confidence = detection[2]
    if confidence > average_confidence_threshold:
        xmin = int(detection[3] * original_image.shape[1])
        ymin = int(detection[4] * original_image.shape[0])
        xmax = int(detection[5] * original_image.shape[1])
        ymax = int(detection[6] * original_image.shape[0])
        detected_faces.append([xmin, ymin, xmax, ymax])
        cv2.rectangle(original_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)


result_image_path = "testimages/result/7.jpg"
cv2.imwrite(result_image_path, original_image)


print(f"Number of students  detected: {len(detected_faces)}")

if len(detected_faces) > 5:
    print("Fan turned on")
else:
    print("Fan turned off")

Number of students  detected: 6
Fan turned on
